In [ ]:
def display_snake(snake_position,display):
    for position in snake_position:
        pygame.draw.rect(display,(255,0,0),pygame.Rect(position[0],position[1],10,10))

def display_apple(apple_position,display):
    pygame.draw.rect(display,(0,255,0),pygame.Rect(apple_position[0], apple_position[1],10,10))

def starting_positions():
    snake_start = [100,100]
    snake_position = [[100,100],[90,100],[80,100]]
    apple_position = [random.randrange(1,50)*10,random.randrange(1,50)*10]
    score = 3
    
    return snake_start, snake_position, apple_position, score

def apple_distance_from_snake(apple_position, snake_position):
    return np.linalg.norm(np.array(apple_position)-np.array(snake_position[0]))

def generate_snake(snake_start, snake_position, apple_position, button_direction, score):

    if button_direction == 1:
        snake_start[0] += 10
    elif button_direction == 0:
        snake_start[0] -= 10
    elif button_direction == 2:
        snake_start[1] += 10
    else:
        snake_start[1] -= 10
        
    if snake_start == apple_position:
        apple_position, score = collision_with_apple(apple_position, score)
        snake_position.insert(0,list(snake_start))

    else:
        snake_position.insert(0,list(snake_start))
        snake_position.pop()
        
    return snake_position, apple_position, score

def collision_with_apple(apple_position, score):
    apple_position = [random.randrange(1,50)*10,random.randrange(1,50)*10]
    score += 1
    return apple_position, score

def collision_with_boundaries(snake_start):
    if snake_start[0]>=500 or snake_start[0]<0 or snake_start[1]>=500 or snake_start[1]<0 :
        return 1
    else:
        return 0

def collision_with_self(snake_start,snake_position):
    #snake_start = snake_position[0]
    if snake_start in snake_position[1:]:
        return 1
    else:
        return 0

def blocked_directions(snake_position):
    current_direction_vector = np.array(snake_position[0])-np.array(snake_position[1])
    
    left_direction_vector = np.array([current_direction_vector[1], -current_direction_vector[0]])
    right_direction_vector = np.array([-current_direction_vector[1], current_direction_vector[0]])
    
    is_front_blocked = is_direction_blocked(snake_position, current_direction_vector)
    is_left_blocked = is_direction_blocked(snake_position, left_direction_vector)
    is_right_blocked = is_direction_blocked(snake_position, right_direction_vector)
    
    return current_direction_vector, is_front_blocked, is_left_blocked, is_right_blocked

def is_direction_blocked(snake_position, current_direction_vector):
    next_step = snake_position[0]+ current_direction_vector
    snake_start = snake_position[0]
    if collision_with_boundaries(next_step) == 1 or collision_with_self(next_step.tolist(),snake_position) == 1:
        return 1
    else:
        return 0

def generate_random_direction(snake_position, angle_with_apple):
    direction = 0
    if angle_with_apple > 0 :
        direction = 1
    elif angle_with_apple < 0 :
        direction = -1
    else:
        direction = 0
        
    return direction_vector(snake_position, angle_with_apple, direction)

def direction_vector(snake_position, angle_with_apple, direction):
    
    current_direction_vector = np.array(snake_position[0])-np.array(snake_position[1])
    left_direction_vector = np.array([current_direction_vector[1],-current_direction_vector[0]])
    right_direction_vector = np.array([-current_direction_vector[1], current_direction_vector[0]])
    
    new_direction = current_direction_vector 
    
    if direction == -1:
        new_direction = left_direction_vector
    if direction == 1:
        new_direction = right_direction_vector
    
    button_direction = generate_button_direction(new_direction)
    
    return direction, button_direction
        
        
def generate_button_direction(new_direction):
    button_direction = 0
    if new_direction.tolist() == [10,0]:
        button_direction = 1
    elif new_direction.tolist() == [-10,0]:
        button_direction = 0
    elif new_direction.tolist() == [0,10]:
        button_direction = 2
    else:
        button_direction = 3
        
    return button_direction

def angle_with_apple(snake_position, apple_position):
    apple_direction_vector = np.array(apple_position)-np.array(snake_position[0])
    snake_direction_vector = np.array(snake_position[0])-np.array(snake_position[1])
    
    norm_of_apple_direction_vector = np.linalg.norm(apple_direction_vector)
    norm_of_snake_direction_vector = np.linalg.norm(snake_direction_vector)
    if norm_of_apple_direction_vector == 0:
        norm_of_apple_direction_vector = 10
    if norm_of_snake_direction_vector == 0:
        norm_of_snake_direction_vector = 10
        
    apple_direction_vector_normalized = apple_direction_vector/norm_of_apple_direction_vector
    snake_direction_vector_normalized = snake_direction_vector/norm_of_snake_direction_vector
    angle = math.atan2(apple_direction_vector_normalized[1] * snake_direction_vector_normalized[0] - apple_direction_vector_normalized[0] * snake_direction_vector_normalized[1], apple_direction_vector_normalized[1] * snake_direction_vector_normalized[1] + apple_direction_vector_normalized[0] * snake_direction_vector_normalized[0]) / math.pi
    return angle, snake_direction_vector

In [ ]:
def play_game(snake_start, snake_position, apple_position, button_direction, score,display,clock):
    crashed = False
    while crashed is not True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                crashed = True
        display.fill((255,255,255))
    
        display_apple(apple_position,display)
        display_snake(snake_position,display)
    
        snake_position, apple_position, score = generate_snake(snake_start, snake_position, apple_position, button_direction, score)
        pygame.display.set_caption("SCORE: "+str(score))
        pygame.display.update()
        clock.tick(5000)
    
        return snake_position, apple_position, score

In [ ]:
def generate_training_data(display,clock):
    max_score = 3
    avg_score = 0
    training_data_x = []
    training_data_y = []
    training_games = 1000
    steps_per_game = 2000
    
    for _ in tqdm(range(training_games)):
        snake_start, snake_position, apple_position, score = starting_positions()
        prev_apple_distance = apple_distance_from_snake(apple_position, snake_position)
        prev_score = score
        
        for _ in range(steps_per_game):
            angle, snake_direction_vector = angle_with_apple(snake_position, apple_position)
            direction, button_direction = generate_random_direction(snake_position, angle)
            current_direction_vector, is_front_blocked, is_left_blocked ,is_right_blocked = blocked_directions(snake_position)
            
            if direction == -1:
                if is_left_blocked == 1:
                    if is_front_blocked == 1 and is_right_blocked == 0:
                        direction, button_direction = direction_vector(snake_position, angle_with_apple, 1)
                        training_data_y.append([0,0,1])
                    elif is_front_blocked == 0 and is_right_blocked == 1:
                        direction, button_direction = direction_vector(snake_position, angle_with_apple, 0)
                        training_data_y.append([0,1,0])
                    elif is_front_blocked == 0 and is_right_blocked == 0:
                        direction, button_direction = direction_vector(snake_position, angle_with_apple, 1) 
                        training_data_y.append([0,0,1])
                    else:
                        break              
                else:
                    training_data_y.append([1,0,0])
     
            elif direction == 0:
                if is_front_blocked == 1:
                    if is_left_blocked == 1 and is_right_blocked == 0:
                        direction, button_direction = direction_vector(snake_position, angle_with_apple, 1)
                        training_data_y.append([0,0,1])
                    elif is_left_blocked == 0 and is_right_blocked == 1:
                        direction, button_direction = direction_vector(snake_position, angle_with_apple, -1)
                        training_data_y.append([1,0,0])
                    elif is_left_blocked == 0 and is_right_blocked == 0:
                        training_data_y.append([0,0,1])
                        direction, button_direction = direction_vector(snake_position, angle_with_apple, 1)
                    else:    
                        break            
                else:
                    training_data_y.append([0,1,0])
            else:
                if is_right_blocked == 1:
                    if is_left_blocked == 1 and is_front_blocked == 0:
                        direction, button_direction = direction_vector(snake_position, angle_with_apple, 0)
                        training_data_y.append([0,1,0])
                    elif is_left_blocked == 0 and is_front_blocked == 1:
                        direction, button_direction = direction_vector(snake_position, angle_with_apple, -1)
                        training_data_y.append([1,0,0])
                    elif is_left_blocked == 0 and is_front_blocked == 0:
                        direction, button_direction = direction_vector(snake_position, angle_with_apple, -1)
                        training_data_y.append([1,0,0])               
                    else:  
                        break
                else:
                    training_data_y.append([0,0,1])

            
            
            
            training_data_x.append([snake_position[0][0],snake_position[0][1],snake_direction_vector[0],snake_direction_vector[1],apple_position[0],apple_position[1],is_left_blocked,is_front_blocked, is_right_blocked])
                
            snake_position, apple_position, score = play_game(snake_start, snake_position, apple_position, button_direction, score,display,clock)   

            if score > max_score:
                max_score = score

        avg_score += score
        
    print("Maximum score during training is: ", max_score)
    print("Average score during training is", avg_score/1000)
    return training_data_x, training_data_y


In [ ]:
def run_game_with_ML(model,display,clock):
    max_score = 3
    avg_score = 0
    test_games = 1000
    steps_per_game = 2000
    
    for _ in tqdm(range(test_games)):
        snake_start, snake_position, apple_position, score = starting_positions()
        
        count_same_direction = 0
        prev_direction = 0

        for _ in range(steps_per_game):
            current_direction_vector, is_front_blocked, is_left_blocked ,is_right_blocked = blocked_directions(snake_position)
            angle, snake_direction_vector = angle_with_apple(snake_position, apple_position)
            predictions = []
            predicted_direction = np.argmax(np.array(model.predict(np.array([snake_position[0][0],snake_position[0][1],snake_direction_vector[0],snake_direction_vector[1],apple_position[0],apple_position[1],is_left_blocked,is_front_blocked, is_right_blocked]).reshape(-1,9))))-1
            
            
            if predicted_direction == prev_direction:
                count_same_direction+=1
            else:
                count_same_direction = 0
                prev_direction = predicted_direction
            
            new_direction = np.array(snake_position[0]) - np.array(snake_position[1])
            if predicted_direction == -1:
                new_direction = np.array([new_direction[1],-new_direction[0]])
            if predicted_direction == 1:
                new_direction = np.array([-new_direction[1],new_direction[0]])
            
            button_direction = generate_button_direction(new_direction)
            
            next_step = snake_position[0]+ current_direction_vector
            if collision_with_boundaries(snake_position[0]) == 1 or collision_with_self(next_step.tolist(), snake_position) == 1:
                break
                
            snake_position, apple_position, score = play_game(snake_start, snake_position, apple_position, button_direction, score,display,clock)  
            
                                         
            if score > max_score:
                max_score = score

        avg_score += score
            
    return max_score, avg_score/1000


In [ ]:
def train_model(parameters):

    units_per_layer = parameters[0]
    layers = parameters[1]
    activation = parameters[2]
    optimizer = parameters[3]
    
    model = Sequential()
    model.add(Dense(units=9,input_dim=9))

    for _ in range(layers):
        model.add(Dense(units=units_per_layer, activation=activation))
        
    model.add(Dense(units = 3,  activation = 'softmax'))
    
    return model

In [ ]:
def hyperparameters():
    parameters = []
    units_per_layer = [20,25,30,35,40]
    layers = [1, 2, 3, 4, 5,6,7]
    activation = ['relu','softmax','sigmoid','tanh']
    optimizer = ['adam','rmsprop']
    parameters.append(units_per_layer)
    parameters.append(layers)
    parameters.append(activation)
    parameters.append(optimizer)
    
    return parameters
    

In [ ]:
def generate_population(size):
    parameters = hyperparameters()
    
    population = []
    i=0   
    while i < size:
        chrosome = [random.choice(parameters[0]), random.choice(parameters[1]),
                    random.choice(parameters[2]), random.choice(parameters[3])]
        if chrosome not in population:
            population.append(chrosome)
            i+=1
    return population
    

In [ ]:
def children_population(size, population):
    
    child_population = []
    k = 0
    population_size = len(population)
    while k < size :
        i, j = random.sample(range(0, population_size), 2)
        parent1 = population[i]
        parent2 = population[j]
         
        child = new_child(parent1, parent2) 
        if child not in child_population and child not in population:
            child_population.append(child)
            k +=1
            
    for newest_child in child_population:
        population.append(newest_child)
        
    return population

In [ ]:
def new_child(parent1, parent2):
    child = []
    parent_size = len(parent1)
    i=0
    while i<parent_size:
        child.append(random.choice([parent1[i],parent2[i]]))
        i += 1
    
    return child
    

In [ ]:
def mutation(population):
    parameters = hyperparameters()
    for chromosome in population:
        if random.random() < 0.1 :
            key = random.choice([0,1,2,3])
            parameters = hyperparameters()
            mutate_key = random.choice(parameters[key])
            chromosome[key] = mutate_key
    
    return population
            
        

In [ ]:
import pygame
import random
import time
import math
from tqdm import tqdm
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from operator import itemgetter

display_width = 500
display_height = 500
green = (0,255,0)
red = (255,0,0)
black = (0,0,0)
white = (255,255,255)

pygame.init()
display=pygame.display.set_mode((display_width,display_height))
clock=pygame.time.Clock()

'''
LEFT -> button_direction = 0
RIGHT -> button_direction = 1
DOWN ->button_direction = 2
UP -> button_direction = 3
'''
training_data_x, training_data_y = generate_training_data(display,clock)

In [ ]:
generations = 10
population_size = 20

population = generate_population(population_size)
population_best = []

for _ in tqdm(range(generations)):
    
    population_with_avg_score = []
    for chromosome in population:
        
        model = train_model(chromosome)
        model.compile(loss='mean_squared_error', optimizer=chromosome[3], metrics=['accuracy'])
        model.fit((np.array(training_data_x).reshape(-1,9)),( np.array(training_data_y).reshape(-1,3)), batch_size = 256,epochs= 3)
        
        max_score, avg_score = run_game_with_ML(model,display,clock)
        print("maximum score: ",max_score)
        print("average_score: ",avg_score)
        print(chromosome)
        chromosome.append(avg_score)
        chromosome.append(max_score)
        population_with_avg_score.append(chromosome)
    
    
    for best in population_best:
        population_with_avg_score.append(best)
        
    population_with_avg_score.sort(key = itemgetter(4),reverse=True)

    
    del population[:]
    del population_best[:]

    

    for i in range(5):
        population_best.append(population_with_avg_score[i])
        population.append(population_with_avg_score[i][:4])
        
    
    
    j, k, l = random.sample(range(5, 20),3)
    population_best.append(population_with_avg_score[j])
    population_best.append(population_with_avg_score[k])
    population_best.append(population_with_avg_score[l])

    population.append(population_with_avg_score[j][:4])
    population.append(population_with_avg_score[k][:4])
    population.append(population_with_avg_score[l][:4])

    
    
    population = children_population(12, population)

    population = mutation(population)
    
    population = population[8:]
    
    print("Best five with random three from population:",  population_best)
  
    
print("Best five with random three from population:",  population_best)